In [1]:
import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from matplotlib import interactive
interactive(True)
%matplotlib qt


import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import my_model

Load in data

In [2]:
def show_statistic(l):
    max_value = max(l)
    min_value = min(l)
    mean = sum(l)/len(l)
    var = sum([((x - mean) ** 2) for x in l]) / len(l)
    std = var**0.5
    print("min:", min_value)
    print("max:", max_value)
    print("mean:", mean)
    print("std:", std)

def normalization(l):
    max_value = max(l)
    min_value = min(l)
    mean = sum(l)/len(l)
    var = sum([((x - mean) ** 2) for x in l]) / len(l)
    std = var**0.5
    # print("min:", min_value)
    # print("max:", max_value)
    # print("mean:", mean)
    # print("std:", std)
    
    for i in range(len(l)):
        l[i] = (l[i] - mean)/std
    
    return l
    

def load_data(file_name, test_size):
    f = open(file_name)
    df = pd.read_csv(f)
    data = np.array(df[['x_0','x']])
    x = data[:,0].tolist()
    x.append(data[-1, -1])
    # do normalization
    # show_statistic(x)
    x = normalization(x)
    # show_statistic(x)
    
    data = np.array(df[['y_0','y']])
    y = data[:,0].tolist()
    y.append(data[-1, -1])
    # do normalization
    y = normalization(y)

    data = np.array(df[['z_0','z']])
    z = data[:,0].tolist()
    z.append(data[-1, -1])
    # do normalization
    z = normalization(z)
    
    train_set_x = x[:-test_size]
    test_set_x = x[-test_size:]
    train_set_y = y[:-test_size]
    test_set_y = y[-test_size:]
    train_set_z = z[:-test_size]
    test_set_z = z[-test_size:]
    
    return train_set_x, test_set_x, train_set_y, test_set_y, train_set_z, test_set_z

In [3]:
train_set_x_1, test_set_x_1, train_set_y_1, test_set_y_1, train_set_z_1, test_set_z_1 = load_data('data_preprocessing/test_1_training_xyz.txt', 400)
train_set_x_2, test_set_x_2, train_set_y_2, test_set_y_2, train_set_z_2, test_set_z_2 = load_data('data_preprocessing/test_2_training_xyz.txt', 400)
train_set_x_3, test_set_x_3, train_set_y_3, test_set_y_3, train_set_z_3, test_set_z_3 = load_data('data_preprocessing/test_3_training_xyz.txt', 400)
train_set_x_4, test_set_x_4, train_set_y_4, test_set_y_4, train_set_z_4, test_set_z_4 = load_data('data_preprocessing/test_4_training_xyz.txt', 400)
train_set_x_5, test_set_x_5, train_set_y_5, test_set_y_5, train_set_z_5, test_set_z_5 = load_data('data_preprocessing/test_5_training_xyz.txt', 400)

In [4]:
# construct list of input and label pairs
def input_data(seq, ws):
    out = []
    L = len(seq)
    
    for i in range(L - ws):
        window = seq[i:i+ws]
        label = seq[i+ws:i+ws+1]
        out.append((window, label))
        
    return out

def get_tensor(sample_size, data_x, data_y, data_z):
    for i in range(sample_size):
        # construct dataset
        x = np.asarray(data_x[i][0]).reshape(1,-1) # 1 x window_size
        y = np.asarray(data_y[i][0]).reshape(1,-1) # 1 x window_size
        z = np.asarray(data_z[i][0]).reshape(1,-1) # 1 x window_size
        xyz = np.append(np.append(x, y, axis = 0), z, axis = 0) # 3 x window_size
        xyz = np.transpose(xyz) # window_size x 3
        window_size = xyz.shape[0]
        # print("window_size", window_size)
        xyz = xyz.reshape(1, window_size, 3) # 1 x window_size x 3
        if (i == 0):
            prev_xyz = xyz
        else:
            prev_xyz = np.append(prev_xyz, xyz, axis = 0)
        
        #construct label
        x_label = np.asarray(data_x[i][1]).reshape(1,-1) # 1 x 1
        y_label = np.asarray(data_y[i][1]).reshape(1,-1) # 1 x 1
        z_label = np.asarray(data_z[i][1]).reshape(1,-1) # 1 x 1
        xyz_label = np.append(np.append(x_label, y_label, axis = 0), z_label, axis = 0) # 3 x 1
        xyz_label = xyz_label.reshape(1, 3) # 1 x 3 x 1
        if (i == 0):
            prev_xyz_label = xyz_label
        else:
            prev_xyz_label = np.append(prev_xyz_label, xyz_label, axis = 0)
            
    return prev_xyz, prev_xyz_label

# make train_dataset
def construct_train_test_tensor(train_set_x, train_set_y, train_set_z, test_set_x, test_set_y, test_set_z, window_size):
    # sequence to data sample
    train_data_x = input_data(train_set_x, window_size)
    train_data_y = input_data(train_set_y, window_size)
    train_data_z = input_data(train_set_z, window_size)
    test_data_x = input_data(test_set_x, window_size)
    test_data_y = input_data(test_set_y, window_size)
    test_data_z = input_data(test_set_z, window_size)
    
    
    # reconstruct train/test dataset and label
    sample_size = len(train_data_x)
    train_dataset, train_label = get_tensor(sample_size, train_data_x, train_data_y, train_data_z)
    sample_size = len(test_data_x)
    test_dataset, test_label = get_tensor(sample_size, test_data_x, test_data_y, test_data_z)
    
    train_dataset = train_dataset.astype("float32")
    train_label = train_label.astype("float32")
    test_dataset = test_dataset.astype("float32")
    test_label = test_label.astype("float32")
    
    return train_dataset, train_label, test_dataset, test_label

In [5]:
window_size = 10
train_dataset_1, train_label_1, test_dataset_1, test_label_1 = construct_train_test_tensor(train_set_x_1,
                                                                                           train_set_y_1,
                                                                                           train_set_z_1,
                                                                                           test_set_x_1,
                                                                                           test_set_y_1,
                                                                                           test_set_z_1,
                                                                                           window_size)

train_dataset_2, train_label_2, test_dataset_2, test_label_2 = construct_train_test_tensor(train_set_x_2,
                                                                                           train_set_y_2,
                                                                                           train_set_z_2,
                                                                                           test_set_x_2,
                                                                                           test_set_y_2,
                                                                                           test_set_z_2,
                                                                                           window_size)

train_dataset_3, train_label_3, test_dataset_3, test_label_3 = construct_train_test_tensor(train_set_x_3,
                                                                                           train_set_y_3,
                                                                                           train_set_z_3,
                                                                                           test_set_x_3,
                                                                                           test_set_y_3,
                                                                                           test_set_z_3,
                                                                                           window_size)

train_dataset_4, train_label_4, test_dataset_4, test_label_4 = construct_train_test_tensor(train_set_x_4,
                                                                                           train_set_y_4,
                                                                                           train_set_z_4,
                                                                                           test_set_x_4,
                                                                                           test_set_y_4,
                                                                                           test_set_z_4,
                                                                                           window_size)

train_dataset_5, train_label_5, test_dataset_5, test_label_5 = construct_train_test_tensor(train_set_x_5,
                                                                                           train_set_y_5,
                                                                                           train_set_z_5,
                                                                                           test_set_x_5,
                                                                                           test_set_y_5,
                                                                                           test_set_z_5,
                                                                                           window_size)

In [6]:
train_dataset_1.dtype

dtype('float32')

In [7]:
train_dataset_1.shape

(2990, 10, 3)

In [8]:
train_label_1.shape

(2990, 3)

In [9]:
test_dataset_1.shape

(390, 10, 3)

In [10]:
test_label_1.shape

(390, 3)

In [11]:
type(train_dataset_1)

numpy.ndarray

In [12]:
# Concatenate tensors
train_dataset = np.concatenate((train_dataset_1,
                                train_dataset_2,
                                train_dataset_3,
                                train_dataset_4,
                                train_dataset_5), axis=0)
train_label = np.concatenate((train_label_1,
                              train_label_2,
                              train_label_3,
                              train_label_4,
                              train_label_5), axis=0)
test_dataset = np.concatenate((test_dataset_1,
                               test_dataset_2,
                               test_dataset_3,
                               test_dataset_4,
                               test_dataset_5), axis=0)
test_label = np.concatenate((test_label_1,
                             test_label_2,
                             test_label_3,
                             test_label_4,
                             test_label_5), axis=0)

In [13]:
train_dataset.dtype

dtype('float32')

In [14]:
train_dataset.shape

(14950, 10, 3)

In [15]:
train_label.shape

(14950, 3)

In [16]:
test_dataset.shape

(1950, 10, 3)

In [17]:
test_label.shape

(1950, 3)

In [18]:
2990*5

14950

In [19]:
390*5

1950

In [20]:
2 x 5 x 5 x 13 x 23 = 14950

SyntaxError: invalid syntax (<ipython-input-20-9979ff4634ab>, line 1)

In [21]:
2 x 5 x 5 x 13 x 3 = 1950

SyntaxError: invalid syntax (<ipython-input-21-de69bcda9ea3>, line 1)

In [22]:
2 * 5 * 5 * 13 * 23

14950

In [23]:
2 * 5 * 5 * 13 * 3

1950

In [24]:
13 * 23

299

In [25]:
13 * 3

39

Prepare dataset

In [26]:
class MyTrainDataSet(Dataset):
    def __init__(self, train_dataset, train_label):
        self.train_dataset = train_dataset
        self.train_label = train_label
        
    def __len__(self):
        return len(self.train_dataset)

    def __getitem__(self, idx):
        seq = self.train_dataset[idx]
        label = self.train_label[idx]
        
        return [seq, label]


class MyTestDataSet(Dataset):
    def __init__(self, test_dataset, test_label):
        self.test_dataset = test_dataset
        self.test_label = test_label

    def __len__(self):
        return len(self.test_dataset)

    def __getitem__(self, idx):
        seq = self.test_dataset[idx]
        label = self.test_label[idx]

        return [seq, label]

In [27]:
train_set = MyTrainDataSet(train_dataset, train_label)
print(len(train_set))
valid_set = MyTestDataSet(test_dataset, test_label)
print(len(valid_set))

14950
1950


In [28]:
type(train_set.train_dataset)

numpy.ndarray

In [29]:
# batch_size = 30
batch_size = 50
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
# keep the test data trajectory order
test_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=0) # dont shuffle test data for using continous trajectory later on
# test_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0)

In [30]:
train_loader

In [31]:
for i, (seqs, labels) in enumerate(test_loader):
    print(i, seqs.shape, labels.shape)

0 torch.Size([50, 10, 3]) torch.Size([50, 3])
1 torch.Size([50, 10, 3]) torch.Size([50, 3])
2 torch.Size([50, 10, 3]) torch.Size([50, 3])
3 torch.Size([50, 10, 3]) torch.Size([50, 3])
4 torch.Size([50, 10, 3]) torch.Size([50, 3])
5 torch.Size([50, 10, 3]) torch.Size([50, 3])
6 torch.Size([50, 10, 3]) torch.Size([50, 3])
7 torch.Size([50, 10, 3]) torch.Size([50, 3])
8 torch.Size([50, 10, 3]) torch.Size([50, 3])
9 torch.Size([50, 10, 3]) torch.Size([50, 3])
10 torch.Size([50, 10, 3]) torch.Size([50, 3])
11 torch.Size([50, 10, 3]) torch.Size([50, 3])
12 torch.Size([50, 10, 3]) torch.Size([50, 3])
13 torch.Size([50, 10, 3]) torch.Size([50, 3])
14 torch.Size([50, 10, 3]) torch.Size([50, 3])
15 torch.Size([50, 10, 3]) torch.Size([50, 3])
16 torch.Size([50, 10, 3]) torch.Size([50, 3])
17 torch.Size([50, 10, 3]) torch.Size([50, 3])
18 torch.Size([50, 10, 3]) torch.Size([50, 3])
19 torch.Size([50, 10, 3]) torch.Size([50, 3])
20 torch.Size([50, 10, 3]) torch.Size([50, 3])
21 torch.Size([50, 10, 

In [32]:
for i, (seqs, labels) in enumerate(train_loader):
    if (i == 0):
        print(i, seqs.dtype, labels.dtype)
        print(seqs[0])
        print(labels[0])

0 torch.float32 torch.float32
tensor([[-1.5292,  0.4927,  0.3011],
        [-1.5173,  0.4927,  0.4030],
        [-1.4933,  0.4927,  0.5026],
        [-1.4408,  0.4927,  0.5906],
        [-1.3852,  0.4927,  0.6768],
        [-1.3296,  0.4927,  0.7630],
        [-1.2740,  0.4927,  0.8492],
        [-1.2155,  0.4927,  0.9333],
        [-1.1400,  0.4927,  1.0025],
        [-1.0622,  0.4927,  1.0690]])
tensor([-0.9840,  0.4927,  1.1351])


In [33]:
for i, (seqs, labels) in enumerate(test_loader):
    if (i == 0):
        print(i, seqs.dtype, labels.dtype)
        print(seqs[1])
        print(labels[1])

0 torch.float32 torch.float32
tensor([[ 0.1533, -0.3653,  1.0343],
        [ 0.1890, -0.3653,  1.0158],
        [ 0.2233, -0.3653,  0.9950],
        [ 0.2577, -0.3653,  0.9742],
        [ 0.2920, -0.3653,  0.9533],
        [ 0.3263, -0.3653,  0.9325],
        [ 0.3606, -0.3653,  0.9117],
        [ 0.3949, -0.3653,  0.8908],
        [ 0.4292, -0.3653,  0.8700],
        [ 0.4635, -0.3653,  0.8491]])
tensor([ 0.4978, -0.3653,  0.8283])


In [34]:
torch.cuda.is_available()

True

In [35]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first = True)
        # If your input data is of shape (seq_len, batch_size, features)
        # then you don’t need batch_first=True and your LSTM will give
        # output of shape (seq_len, batch_size, hidden_size).

        # If your input data is of shape (batch_size, seq_len, features)
        # then you need batch_first=True and your LSTM will give
        # output of shape (batch_size, seq_len, hidden_size).
        
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        # self.hidden = (torch.zeros(self.num_layers, 1, hidden_size), torch.zeros(self.num_layers, 1, hidden_size))
        
    def forward(self, x):
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim,
                                      x.size(0), # batch_size, retrieved from batch data x
                                      self.hidden_dim)).cuda()
        else:
            h0 = Variable(torch.zeros(self.layer_dim,
                                      x.size(0), # batch_size, retrieved from batch data x
                                      self.hidden_dim))
        if torch.cuda.is_available():
            c0 = Variable(torch.zeros(self.layer_dim,
                                      x.size(0), # batch_size, retrieved from batch data x
                                      self.hidden_dim)).cuda()
        else:
            c0 = Variable(torch.zeros(self.layer_dim,
                                      x.size(0), # batch_size, retrieved from batch data x
                                      self.hidden_dim))
        # print("x.size(0)", x.size(0))
        
        out, (hn, cn) = self.lstm(x, (h0, c0))
        # x is (batch_size, seq_len, features)
        
        out = self.fc(out[:, -1, :])
        
        return out
    

In [43]:
input_dim = 3
hidden_dim = 100
layer_dim = 1
output_dim = 3
model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)

if torch.cuda.is_available():
    model.cuda()

In [44]:
criterion = nn.MSELoss()

In [45]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

Check parameters

In [46]:
len(list(model.parameters()))

6

In [47]:
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

torch.Size([400, 3])
torch.Size([400, 100])
torch.Size([400])
torch.Size([400])
torch.Size([3, 100])
torch.Size([3])


Training

In [48]:
seq_dim = 10 # = window_size

num_epochs = 1600 # 400 # 200 will overfit, 100 is good, see the plot: plt.plot(train_loss[20:]) and plt.plot(test_loss[20:])

train_loss = []
test_loss = []



for epoch in range(num_epochs):
    total_train_loss = 0.0
    train_batch = 0
    for i, (seqs, labels) in enumerate(train_loader):
        # print("train: ", i)
        if torch.cuda.is_available():
            # print(seqs.shape)
            # print(seqs[0])
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            # print(labels.shape)
            # print(seqs[0])
            # seqs = Variable(seqs)
            labels = Variable(labels.cuda())
        else:
            # print(seqs.shape)
            # print(seqs[0])
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))
            # print(labels.shape)
            # print(seqs[0])
            # seqs = Variable(seqs)
            labels = Variable(labels)
        
        
        optimizer.zero_grad()
        # outputs = model(seqs.float())
        outputs = model(seqs)
        # print(outputs.is_cuda)
        # print(labels.is_cuda)
        # print(outputs.shape)
        # print(outputs.dtype)
        
        # loss = criterion(outputs, labels.float())
        loss = criterion(outputs, labels)
        total_train_loss += loss.data.item()
        
        loss.backward()
        optimizer.step()
        
        train_batch = i + 1;
        
        # print("loss: ", loss.data)
    
    total_test_loss = 0.0
    test_batch = 0
    # test_seq = []
    test_pred = []
    # test_gt = []
    for i, (seqs, labels) in enumerate(test_loader):
        # print("test: ", i)
        if torch.cuda.is_available():
            seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
            labels = Variable(labels.cuda())
        else:
            seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        outputs = model(seqs)
        # print(i, outputs.shape)
        # print(i, outputs.is_cuda)
        loss = criterion(outputs, labels)
        total_test_loss += loss.data.item()
        # test_gt.append(labels)
        test_pred.append(outputs)
        # test_seq.append(seqs)
        
        test_batch = i + 1
    
    print("train batch: ", train_batch)
    print("test batch: ", test_batch)
    train_loss.append(total_train_loss/train_batch)
    test_loss.append(total_test_loss/test_batch)
    # train_loss.append(total_train_loss)
    # test_loss.append(total_test_loss)
    print("epoch: {}, train_loss: {}, test_loss: {}".format(epoch, total_train_loss/train_batch, total_test_loss/test_batch))
        
        

train batch:  299
test batch:  39
epoch: 0, train_loss: 0.6836076390783125, test_loss: 0.39737762396152204
train batch:  299
test batch:  39
epoch: 1, train_loss: 0.1502211430549223, test_loss: 0.07148451902694666
train batch:  299
test batch:  39
epoch: 2, train_loss: 0.039401816328781904, test_loss: 0.04599454452415021
train batch:  299
test batch:  39
epoch: 3, train_loss: 0.03342713529935549, test_loss: 0.04248273972039803
train batch:  299
test batch:  39
epoch: 4, train_loss: 0.031489930093836625, test_loss: 0.04019784139325985
train batch:  299
test batch:  39
epoch: 5, train_loss: 0.029852093465277382, test_loss: 0.03802041957477251
train batch:  299
test batch:  39
epoch: 6, train_loss: 0.028415322908041468, test_loss: 0.03638940070493099
train batch:  299
test batch:  39
epoch: 7, train_loss: 0.02713776745811653, test_loss: 0.03478986100974278
train batch:  299
test batch:  39
epoch: 8, train_loss: 0.02599501683934277, test_loss: 0.033326467173771024
train batch:  299
test ba

train batch:  299
test batch:  39
epoch: 75, train_loss: 0.0058942066301532516, test_loss: 0.007144470475069307
train batch:  299
test batch:  39
epoch: 76, train_loss: 0.005812933682510785, test_loss: 0.007022267689395589
train batch:  299
test batch:  39
epoch: 77, train_loss: 0.005733191319912622, test_loss: 0.0069111960705739856
train batch:  299
test batch:  39
epoch: 78, train_loss: 0.00565888985826394, test_loss: 0.006856131908791259
train batch:  299
test batch:  39
epoch: 79, train_loss: 0.005587566971395751, test_loss: 0.006778335467037649
train batch:  299
test batch:  39
epoch: 80, train_loss: 0.0055187876763781625, test_loss: 0.006666354263404337
train batch:  299
test batch:  39
epoch: 81, train_loss: 0.005452191104012912, test_loss: 0.00657352866181053
train batch:  299
test batch:  39
epoch: 82, train_loss: 0.005389469571763581, test_loss: 0.0064798325930053415
train batch:  299
test batch:  39
epoch: 83, train_loss: 0.005329761182247695, test_loss: 0.006416450481666312

train batch:  299
test batch:  39
epoch: 149, train_loss: 0.003862159191568485, test_loss: 0.004118191412626169
train batch:  299
test batch:  39
epoch: 150, train_loss: 0.003851302908764635, test_loss: 0.004160421557921486
train batch:  299
test batch:  39
epoch: 151, train_loss: 0.003842565909271007, test_loss: 0.004106271454545025
train batch:  299
test batch:  39
epoch: 152, train_loss: 0.00383275411310559, test_loss: 0.004082031274148717
train batch:  299
test batch:  39
epoch: 153, train_loss: 0.003823724812648904, test_loss: 0.004059231299041317
train batch:  299
test batch:  39
epoch: 154, train_loss: 0.0038140275648773266, test_loss: 0.0040657277016016916
train batch:  299
test batch:  39
epoch: 155, train_loss: 0.003805832657807298, test_loss: 0.004041018850283506
train batch:  299
test batch:  39
epoch: 156, train_loss: 0.0037968551119428166, test_loss: 0.0040077113013607096
train batch:  299
test batch:  39
epoch: 157, train_loss: 0.00378848233335748, test_loss: 0.003989396

train batch:  299
test batch:  39
epoch: 222, train_loss: 0.003388514577949974, test_loss: 0.0032634503634956977
train batch:  299
test batch:  39
epoch: 223, train_loss: 0.003383715829755924, test_loss: 0.003277634471977273
train batch:  299
test batch:  39
epoch: 224, train_loss: 0.003379692920377746, test_loss: 0.0032829566737816026
train batch:  299
test batch:  39
epoch: 225, train_loss: 0.0033763150472729702, test_loss: 0.0032536842157312026
train batch:  299
test batch:  39
epoch: 226, train_loss: 0.003371420499392128, test_loss: 0.0032590257503146017
train batch:  299
test batch:  39
epoch: 227, train_loss: 0.0033674671849622125, test_loss: 0.0032392293203604193
train batch:  299
test batch:  39
epoch: 228, train_loss: 0.0033625383471235335, test_loss: 0.003224626006648088
train batch:  299
test batch:  39
epoch: 229, train_loss: 0.0033590708349996484, test_loss: 0.003225000124821702
train batch:  299
test batch:  39
epoch: 230, train_loss: 0.0033546874891845105, test_loss: 0.0

train batch:  299
test batch:  39
epoch: 295, train_loss: 0.0031307184158428807, test_loss: 0.00285284624409635
train batch:  299
test batch:  39
epoch: 296, train_loss: 0.0031287325513050784, test_loss: 0.002861007865972053
train batch:  299
test batch:  39
epoch: 297, train_loss: 0.0031252219896517495, test_loss: 0.0028638258958474183
train batch:  299
test batch:  39
epoch: 298, train_loss: 0.0031220647124853918, test_loss: 0.0028483702753407834
train batch:  299
test batch:  39
epoch: 299, train_loss: 0.0031189587034197853, test_loss: 0.00285190417810391
train batch:  299
test batch:  39
epoch: 300, train_loss: 0.0031165936669758382, test_loss: 0.0028504972791639515
train batch:  299
test batch:  39
epoch: 301, train_loss: 0.0031137037817871565, test_loss: 0.0028353927165228063
train batch:  299
test batch:  39
epoch: 302, train_loss: 0.0031104202239764824, test_loss: 0.002830306173284323
train batch:  299
test batch:  39
epoch: 303, train_loss: 0.003106951834844195, test_loss: 0.0

...

train batch:  299
test batch:  39
epoch: 1241, train_loss: 0.0019085105953591697, test_loss: 0.0015102211525514482
train batch:  299
test batch:  39
epoch: 1242, train_loss: 0.0019081673725238723, test_loss: 0.0014989465969101263
train batch:  299
test batch:  39
epoch: 1243, train_loss: 0.0019071242432530508, test_loss: 0.0014976886704038733
train batch:  299
test batch:  39
epoch: 1244, train_loss: 0.0019070617786945813, test_loss: 0.0015152266466593107
train batch:  299
test batch:  39
epoch: 1245, train_loss: 0.001906479172473602, test_loss: 0.0015059366823362114
train batch:  299
test batch:  39
epoch: 1246, train_loss: 0.0019058363051169293, test_loss: 0.001511075580007552
train batch:  299
test batch:  39
epoch: 1247, train_loss: 0.0019052630840539061, test_loss: 0.0015031336969024357
train batch:  299
test batch:  39
epoch: 1248, train_loss: 0.0019042688888345025, test_loss: 0.0015018301406272282
train batch:  299
test batch:  39
epoch: 1249, train_loss: 0.0019035318205062998, 

train batch:  299
test batch:  39
epoch: 1313, train_loss: 0.0018652834996493865, test_loss: 0.0014661474887338371
train batch:  299
test batch:  39
epoch: 1314, train_loss: 0.0018647168204266216, test_loss: 0.0014684184518559144
train batch:  299
test batch:  39
epoch: 1315, train_loss: 0.0018634972673370487, test_loss: 0.0014788434332914245
train batch:  299
test batch:  39
epoch: 1316, train_loss: 0.001863409384865513, test_loss: 0.0014814845837043743
train batch:  299
test batch:  39
epoch: 1317, train_loss: 0.0018630221657502468, test_loss: 0.001462935513233951
train batch:  299
test batch:  39
epoch: 1318, train_loss: 0.0018622754317382038, test_loss: 0.0014712514545326717
train batch:  299
test batch:  39
epoch: 1319, train_loss: 0.0018614566422388407, test_loss: 0.0014615834044003023
train batch:  299
test batch:  39
epoch: 1320, train_loss: 0.001861246014216348, test_loss: 0.0014653242609571093
train batch:  299
test batch:  39
epoch: 1321, train_loss: 0.0018603936732080096, t

train batch:  299
test batch:  39
epoch: 1385, train_loss: 0.0018250896583932661, test_loss: 0.0014338378530691187
train batch:  299
test batch:  39
epoch: 1386, train_loss: 0.0018240748598519869, test_loss: 0.0014344147630953768
train batch:  299
test batch:  39
epoch: 1387, train_loss: 0.0018237841495252012, test_loss: 0.0014372548450923299
train batch:  299
test batch:  39
epoch: 1388, train_loss: 0.0018230796224915774, test_loss: 0.0014277953350989637
train batch:  299
test batch:  39
epoch: 1389, train_loss: 0.0018227484796926642, test_loss: 0.001427414762497975
train batch:  299
test batch:  39
epoch: 1390, train_loss: 0.0018222223576796817, test_loss: 0.0014318224226978298
train batch:  299
test batch:  39
epoch: 1391, train_loss: 0.0018212285770195223, test_loss: 0.0014212073551905819
train batch:  299
test batch:  39
epoch: 1392, train_loss: 0.0018213824721374067, test_loss: 0.0014261525791833918
train batch:  299
test batch:  39
epoch: 1393, train_loss: 0.0018206969528251878,

train batch:  299
test batch:  39
epoch: 1457, train_loss: 0.001787257316784332, test_loss: 0.0014009507648528924
train batch:  299
test batch:  39
epoch: 1458, train_loss: 0.0017870914518644716, test_loss: 0.00140434873257898
train batch:  299
test batch:  39
epoch: 1459, train_loss: 0.0017861414605940078, test_loss: 0.001412845810894699
train batch:  299
test batch:  39
epoch: 1460, train_loss: 0.001786011343256938, test_loss: 0.0014008502878893453
train batch:  299
test batch:  39
epoch: 1461, train_loss: 0.0017852512612938102, test_loss: 0.0014024039523158139
train batch:  299
test batch:  39
epoch: 1462, train_loss: 0.001784850267501062, test_loss: 0.0013877852304572931
train batch:  299
test batch:  39
epoch: 1463, train_loss: 0.001784350293771018, test_loss: 0.0013970660285663027
train batch:  299
test batch:  39
epoch: 1464, train_loss: 0.0017841363412929218, test_loss: 0.0013983411714807749
train batch:  299
test batch:  39
epoch: 1465, train_loss: 0.001783614287099935, test_l

train batch:  299
test batch:  39
epoch: 1529, train_loss: 0.0017519428247852987, test_loss: 0.0013913299714122564
train batch:  299
test batch:  39
epoch: 1530, train_loss: 0.001752370964275536, test_loss: 0.0013699705093294585
train batch:  299
test batch:  39
epoch: 1531, train_loss: 0.0017518198547260306, test_loss: 0.001373364487312173
train batch:  299
test batch:  39
epoch: 1532, train_loss: 0.0017512039536256717, test_loss: 0.0013687804485715998
train batch:  299
test batch:  39
epoch: 1533, train_loss: 0.0017508189463528515, test_loss: 0.0013709352370381404
train batch:  299
test batch:  39
epoch: 1534, train_loss: 0.0017500508430236359, test_loss: 0.0013568921130149064
train batch:  299
test batch:  39
epoch: 1535, train_loss: 0.0017496696356987661, test_loss: 0.001365598034727811
train batch:  299
test batch:  39
epoch: 1536, train_loss: 0.0017491513033406061, test_loss: 0.001360122258912993
train batch:  299
test batch:  39
epoch: 1537, train_loss: 0.001748902594562025, tes

In [ ]:
124/(199/39)

In [49]:
starting_epoch = 30
end_epoch = 1600
train_loss_curve, = plt.plot(list(range(starting_epoch, end_epoch)), train_loss[starting_epoch:end_epoch])
test_loss_curve, = plt.plot(list(range(starting_epoch, end_epoch)), test_loss[starting_epoch:end_epoch])
plt.legend([train_loss_curve, test_loss_curve], ['Train Loss', 'Validation Loss'])
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")

Text(0, 0.5, 'MSE Loss')

In [ ]:
list(range(50, len(train_loss)))

In [ ]:
test_pred

In [50]:
PATH = "model_1600.pt"
torch.save(model.state_dict(), PATH)

Loading Model

In [51]:
PATH = "model_1600.pt"
load_model = my_model.LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()
# mmodel = torch.load(PATH)

if torch.cuda.is_available():
    load_model.cuda()

In [52]:
# get test results
seq_dim = 10 # = window_size
input_dim = 3
# test_seq = []
test_predd = []
# test_gt = []
total_test_loss = 0.0
test_batch = 0
for i, (seqs, labels) in enumerate(test_loader):
    if torch.cuda.is_available():
        seqs = Variable(seqs.view(-1, seq_dim, input_dim).cuda())
        labels = Variable(labels.cuda())
    else:
        seqs = Variable(seqs.view(-1, seq_dim, input_dim))
        
    outputs = load_model(seqs)
    loss = criterion(outputs, labels)
    total_test_loss += loss.data.item()
    test_predd.append(outputs)
    test_batch = i + 1
print(total_test_loss/test_batch)

0.0013484270857588854


In [53]:
test_predd[-1]

tensor([[ 0.3591, -0.2594, -1.2198],
        [ 0.1797, -0.2594, -1.2168],
        [ 0.0073, -0.2596, -1.1807],
        [-0.1624, -0.2591, -1.1296],
        [-0.3344, -0.2583, -1.0978],
        [-0.5071, -0.2583, -1.0742],
        [-0.6797, -0.2585, -1.0518],
        [-0.8495, -0.2588, -1.0161],
        [-0.9931, -0.2594, -0.9066],
        [-1.1044, -0.2589, -0.7588],
        [-1.1153, -0.2605, -0.5499],
        [-1.1272, -0.2581, -0.3526],
        [-1.1241, -0.2575, -0.1668],
        [-1.0960, -0.2581,  0.0111],
        [-1.0665, -0.2580,  0.1842],
        [-1.0241, -0.2583,  0.3504],
        [-0.8737, -0.2622,  0.4689],
        [-0.7243, -0.2626,  0.5871],
        [-0.5762, -0.2622,  0.6942],
        [-0.4162, -0.2623,  0.7744],
        [-0.2332, -0.2630,  0.7990],
        [-0.0428, -0.2637,  0.7634],
        [ 0.1369, -0.2643,  0.6991],
        [ 0.3093, -0.2647,  0.6592],
        [ 0.4671, -0.2642,  0.7472],
        [ 0.6353, -0.2610,  0.7547],
        [ 0.8032, -0.2604,  0.6757],
 

In [54]:
test_label_1

array([[ 0.4635323 , -0.36533475,  0.8491472 ],
       [ 0.4978454 , -0.36533475,  0.82831156],
       [ 0.5321585 , -0.36533475,  0.80747604],
       ...,
       [-0.86375517, -1.7444978 , -0.03705572],
       [-0.8341637 , -1.7444978 , -0.01024055],
       [-0.8022799 , -1.7444978 ,  0.0139663 ]], dtype=float32)

Analyze the distribution of ground true around predication

In [56]:
# restore testing data
test_seq = []
test_gt = []
for i, (seqs, labels) in enumerate(test_loader):
    test_gt.append(labels)
    test_seq.append(seqs)

In [57]:
for i in range(len(test_pred)):
    if (i == 0):
        test = test_seq[i].numpy()
        gt = test_gt[i].numpy()
        pred = test_pred[i].cpu().detach().numpy()
    else:
        test = np.append(test, test_seq[i].numpy(), axis = 0)
        gt = np.append(gt, test_gt[i].numpy(), axis = 0)
        pred = np.append(pred, test_pred[i].cpu().detach().numpy(), axis = 0)

In [ ]:
test.shape

In [ ]:
gt.shape

In [ ]:
pred.shape

In [ ]:
test[-1]

In [ ]:
gt[-1]

In [ ]:
pred[-1]

In [ ]:
d = test[0][-1,:].reshape(3, -1)

In [ ]:
test.shape

In [ ]:
d.shape

In [ ]:
gt[0].shape

In [ ]:
gt.shape

Construct analysis matrix l(0-2 col): last sequence point, g(3-5 col): next ground true point, p(6-8 col): predicted point

In [ ]:
for i in range(test.shape[0]):
    last_point = test[i][-1, :].reshape(1, -1)
    gt_point = gt[i].reshape(1, -1)
    pred_point = pred[i].reshape(1, -1)
    row = np.append(np.append(last_point, gt_point, axis = 1), pred_point, axis = 1)
    if (i == 0):
        analysis = row
    else:
        analysis = np.append(analysis, row, axis = 0)
    

In [ ]:
# get rotation matrix
# note: input vec_1 and vec_2 have to be normalized before passing to the function
def get_rotatin_mat(vec_1, vec_2):
    a,b = vec_1.reshape(3), vec_2.reshape(3)
    v = np.cross(a,b)
    c = np.dot(a,b)
    s = np.linalg.norm(v)
    # print("s", s)
    if (s == 0):
        return np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]])
    I = np.identity(3)
    vXStr = '{} {} {}; {} {} {}; {} {} {}'.format(0, -v[2], v[1], v[2], 0, -v[0], -v[1], v[0], 0)
    k = np.matrix(vXStr)
    r = I + k + np.matmul(k,k) * ((1 -c)/(s**2))
    
    return r

In [ ]:
a = np.array([2, 1, 0])
a = a/np.linalg.norm(a)
b = np.array([4, 5, 6])
b = b/np.linalg.norm(b)
print("a", a)
print("b", b)
m = get_rotatin_mat(a, b)

In [ ]:
np.dot(m, a)

In [ ]:
m.shape

In [ ]:
np.sqrt(np.sum(np.square(b[:])))

In [ ]:
np.linalg.det(m)

In [ ]:
b/np.linalg.norm(b)

In [ ]:
a = np.array([[1,2,3]])
a_n = a/np.linalg.norm(a)

b = np.array([[4,5,6]])
b_n = b/np.linalg.norm(b)

c = np.array([[7,8,9]])
c_n = c/np.linalg.norm(c)

r = get_rotatin_mat(b_n, a_n) # b to a

b_p = np.dot(r, b.reshape(3, -1))
c_p = np.dot(r, c.reshape(3, -1))
c_p = np.asarray(c_p)
c_p_n = c_p/np.linalg.norm(c_p)
print(b.shape)
print("a", a)
print("b", b)
print("c", c)
print("b_p", b_p)
print("c_p", c_p)

print(get_rotatin_mat(c_n, b_n))

print(get_rotatin_mat(c_p_n, a_n))

In [ ]:
d = c_p.reshape(1, -1)
d = np.asarray(d)
d_n = d/np.linalg.norm(d)
d

In [ ]:
a

In [ ]:
d.shape

In [ ]:
a.shape

In [ ]:
b.shape

In [ ]:
get_rotatin_mat(d_n, a)

In [ ]:
np.linalg.norm(b)

In [ ]:
np.linalg.norm(b_p)

In [ ]:
analysis.shape

In [ ]:
# combine all point to the same starting point
# first sequence last direction
# direction = (analysis[0, 0:3] - analysis[0, 3:6]).reshape(1, -1)
direction = (analysis[0, 3:6] - analysis[0, 0:3]).reshape(1, -1)
# normalize
direction = direction/np.linalg.norm(direction)
# first diff between prediction and ground true
diff_correct = (analysis[0, 6:9] - analysis[0, 3:6]).reshape(1, -1)
# print(type(diff_correct))
# print(diff_correct.shape)
for i in range(1, analysis.shape[0]):
    # cur_direction = (analysis[i, 0:3] - analysis[i, 3:6]).reshape(1, -1)
    cur_direction = (analysis[i, 3:6] - analysis[i, 0:3]).reshape(1, -1)
    # normalize
    if (np.linalg.norm(cur_direction) == 0):
        print(i, np.linalg.norm(cur_direction))
    cur_direction = cur_direction/np.linalg.norm(cur_direction)
    cur_diff = (analysis[i, 6:9] - analysis[i, 3:6]).reshape(1, -1)
    # get rotation matrix from cur_direction to direction
    r = get_rotatin_mat(cur_direction, direction)
    # apply rotation matrix to the cur_diff
    cur_diff = np.dot(r, cur_diff.reshape(3, -1))
    cur_diff = cur_diff.reshape(1, -1)
    # matrix type to np array type
    cur_diff = np.asarray(cur_diff)
    # print(cur_diff.shape)
    diff_correct = np.append(diff_correct, cur_diff, axis = 0)
    
    

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(diff_correct[:, 0], diff_correct[:, 1], diff_correct[:, 2])
x = [-direction[0, 0], 0]
y = [-direction[0, 1], 0]
z = [-direction[0, 2], 0]
ax.plot(x, y, z, label='parametric curve', color='r')
# ax.arrow(0, 0, 0.5, 0.5, head_width=0.05, head_length=0.1, fc='k', ec='k')
# ax.quiver(0, 0, 0, -direction[0, 0], -direction[0, 1], -direction[0, 2], length=5, normalize=True, color='r')
# x = np.zeros(10)
# y = np.zeros(10)
# z = np.arange(10)*10 # remove *100 and the arrow heads will reappear.
# dx = np.zeros(10)
# dy = np.arange(10)
# dz = np.zeros(10)
x = np.array([0, -direction[0, 0]])
y = np.array([0, -direction[0, 1]])
z = np.array([0, -direction[0, 2]])
dx = np.array([0, 0])
dy = np.array([0, 0])
dz = np.array([0, 0])
# ax.quiver(x, y, z, dx, dy, dz, length=1)
# ax.quiver(-direction[0, 0], -direction[0, 1], -direction[0, 2], 0, 0, 0, length=100, normalize=True)
ax.set_xlabel("x direction")
ax.set_ylabel("y direction")
ax.set_zlabel("z direction")
plt.show()



In [ ]:
direction

In [ ]:
np.zeros(10)

In [ ]:
view_angle = np.array([[1, 2, 3]])

In [ ]:
view_angle

In [ ]:
# note view_angle must be normalized
def getProjection(view_angle, point):
    v = point - np.array([[0, 0, 0]])
    dist = v[0, 0]*view_angle[0, 0] + v[0, 1]*view_angle[0, 1] + v[0, 2]*view_angle[0, 2]
    projected_point = point - dist*view_angle
    
    return projected_point

In [ ]:
diff_correct.shape

In [ ]:
direction[0, :].shape

In [ ]:
direction[0, :]

In [ ]:
view_angle = np.cross(direction[0, :].reshape(1, -1), np.array([[0, 0, 1]]))

In [ ]:
view_angle

In [ ]:
view_angle = np.cross(direction[0, :].reshape(1, -1), np.array([[0, 0, 1]]))
# normalization
view_angle = view_angle/np.linalg.norm(view_angle)
print(view_angle)
for i in range(diff_correct.shape[0]):
    if (i == 0):
        diff_corrrect_projection = getProjection(view_angle, diff_correct[i, :])
    else:
        diff_corrrect_projection = np.append(diff_corrrect_projection, getProjection(view_angle, diff_correct[i, :]), axis = 0)
    

In [ ]:
diff_corrrect_projection.shape

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(diff_corrrect_projection[:, 0], diff_corrrect_projection[:, 1], diff_corrrect_projection[:, 2])
x = [-direction[0, 0]*10, 0]
y = [-direction[0, 1]*10, 0]
z = [-direction[0, 2]*10, 0]
ax.plot(x, y, z, label='parametric curve', color='r')
ax.set_xlabel("x direction")
ax.set_ylabel("y direction")
ax.set_zlabel("z direction")
plt.show()

In [ ]:
diff_corrrect_projection

In [ ]:
# transform from 3D to 2D
diff_corrrect_projection_2D = np.delete(diff_corrrect_projection, 1, 1)

    

In [ ]:
from sklearn import mixture
from matplotlib.colors import LogNorm
clf = mixture.GaussianMixture(n_components=2, covariance_type='full')
clf.fit(diff_corrrect_projection_2D)

In [ ]:
# display predicted scores by the model as a contour plot
x = np.linspace(-5., 5.)
y = np.linspace(-5., 5.)
X, Y = np.meshgrid(x, y)
XX = np.array([X.ravel(), Y.ravel()]).T
Z = -clf.score_samples(XX)
Z = Z.reshape(X.shape)

CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0),
                 levels=np.logspace(0, 3, 300))
CB = plt.colorbar(CS, shrink=0.8, extend='both')
plt.scatter(diff_corrrect_projection_2D[:, 0], diff_corrrect_projection_2D[:, 1], .8)

x = [-direction[0, 0]*5, 0]
z = [-direction[0, 2]*5, 0]
plt.plot(x, z, label='parametric curve', color='r')

plt.title('Negative log-likelihood predicted by a GMM')
plt.axis('tight')
plt.xlabel("Direction 1")
plt.ylabel("Direction 2")
plt.show()

In [ ]:
n_samples = 300
C = np.array([[0., -0.7], [3.5, .7]])
stretched_gaussian = np.dot(np.random.randn(n_samples, 2), C)

In [ ]:
stretched_gaussian.shape

In [ ]:
diff_corrrect_projection_2D.shape

In [ ]:
direction

In [ ]:
pow(direction[0, 0], 2) + pow(direction[0, 1], 2) + pow(direction[0, 2], 2)

In [ ]:
analysis[2, 0:3] - analysis[2, 3:6]

In [ ]:
analysis[3, 0:3] - analysis[3, 3:6]

In [ ]:
np.linalg.norm([3, 4])

In [ ]:
direction = (analysis[0, 0:3] - analysis[0, 3:6]).reshape(1, -1)
direction = direction/np.linalg.norm(direction)

In [ ]:
(analysis[0, 0:3] - analysis[0, 3:6]).reshape(1, -1)

In [ ]:
(analysis[0, 0:3] - analysis[0, 3:6])

In [ ]:
d = (analysis[1, 0:3] - analysis[1, 3:6])

In [ ]:
d

In [ ]:
d/np.linalg.norm(d)

In [ ]:
pow(0.84804916, 2) + pow(0.5299187, 2)

In [ ]:
analysis[0:3]

In [ ]:
(analysis[0, 3:6] - analysis[0, 0:3])

In [ ]:
diff

In [ ]:
np.asarray([13.259178  ,  0.37397736, 20.36138]) - np.asarray([12.658086  ,  1.293065  ,20.224268])

In [ ]:
for i in range(len(test_pred)):
    if (i == 0):
        test = 
        gt = test_gt[i].cpu().detach().numpy()
        pred = test_pred[i].cpu().detach().numpy()
    else:
        gt = np.append(gt, test_gt[i].cpu().detach().numpy(), axis = 0)
        pred = np.append(pred, test_pred[i].cpu().detach().numpy(), axis = 0)

In [ ]:
gt.shape

In [ ]:
pred.shape

In [ ]:
diff = pred - gt

In [ ]:
diff

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(diff[:, 0], diff[:, 1], diff[:, 2])
plt.show()

In [ ]:
criterion(torch.from_numpy(gt), torch.from_numpy(pred))

In [ ]:
loss = 0
for i in range(len(test_pred)):
    l = criterion(test_gt[i], test_pred[i])
    loss += l

In [ ]:
loss